In [1]:
import glob
import os
import sys
from codalab_eval import semantic_change_segmentation as scs
from codalab_eval import jaccard_index as jaccard
import numpy as np
from PIL import Image

# Check input directories.
submit_dir = '/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/predicts/load_with_tif'
truth_dir = '/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/labels'

if not os.path.isdir(submit_dir):
    print("submit_dir {} doesn't exist".format(submit_dir))
    sys.exit()
if not os.path.isdir(truth_dir):
    print("truth_dir {} doesn't exist".format(truth_dir))
    sys.exit()

# Create output directory.
output_dir = '/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/predicts/check_scs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


pred_dir = submit_dir
gt_dir = truth_dir

def _read_image_as_np(path):
    with open(path, 'rb') as f:
        return np.array(Image.open(f))

def _get_cube_id(filename):
    return filename.split('.')[0][:-11]

IMGS_PER_CUBE = 24

scs_metric = scs.SCSMetric(num_classes=6, ignore_label=255)
miou_metric = jaccard.JaccardIndexMetric(num_classes=6, ignore_label=255)

image_list = sorted(glob.glob(os.path.join(pred_dir, '*.png')))
print(len(image_list))
gt, gt_next = None, None
pred, pred_next = None, None


120


In [3]:
# path_val = "/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/splits/val.txt"
# with open(path_val, 'r') as f:
#     data = f.readlines()
#     all_vals = [d.strip() for d in data]
# all_vals.sort()
# len(all_vals), all_vals


In [4]:
for i in range(len(image_list)):

    # gt_img_name = image_list[i]
    # filename = gt_img_name.split('/')[-1]
    # pred_img_name = os.path.join(pred_dir, filename)
    
    pred_img_name = image_list[i]
    filename = pred_img_name.split('/')[-1]
    gt_img_name = os.path.join(truth_dir, filename)
    
    if not os.path.exists(pred_img_name):
        raise ValueError("Submission segmentation map not found - terminating!\n"
                         "Missing segmentation map: {}".format(filename))
    print("Found corresponding submission file {} for reference file {}"
          "".format(pred_img_name, gt_img_name))

    gt = _read_image_as_np(gt_img_name)
    pred = _read_image_as_np(pred_img_name)

    if gt.shape != pred.shape:
        raise AttributeError("Shapes do not match! Image id {}, Prediction mask {}, "
                             "ground truth mask {}"
                             "".format(os.path.basename(filename),
                                       pred.shape,
                                       gt.shape))

    if (pred.astype(int) >= 6).any():
        raise ValueError(
            "Input {} does not correspond to submission format! All predicted mask values need to be from 0 to 5!!!".format(
                filename))

    cube_id = _get_cube_id(filename)
    miou_metric.update_state(gt, pred)
    if (i % IMGS_PER_CUBE) >= (IMGS_PER_CUBE - 1):
        continue

    # gt_img_name_next = image_list[i + 1]
    # filename = gt_img_name_next.split('/')[-1]
    # pred_img_name_next = os.path.join(pred_dir, filename)
    
    pred_img_name_next = image_list[i + 1]
    filename = pred_img_name_next.split('/')[-1]
    gt_img_name_next = os.path.join(truth_dir, filename)



    if not os.path.exists(pred_img_name_next):
        raise ValueError("Submission segmentation map not found - terminating!\n"
                         "Missing segmentation map: {}".format(filename))
    print("Found corresponding submission file {} for reference file {}"
          "".format(pred_img_name_next, gt_img_name_next))

    gt_next = _read_image_as_np(gt_img_name_next)
    pred_next = _read_image_as_np(pred_img_name_next)

    if gt_next.shape != pred_next.shape:
        raise AttributeError("Shapes for the images do not match! Image id {}, Prediction mask {}, "
                             "ground truth mask {}"
                             "".format(os.path.basename(filename),
                                       pred_next.shape,
                                       gt_next.shape))
    #if ((pred_next.astype(int) >= 6) & pred_next.astype(int)).any():
    if (pred_next.astype(int) >= 6).any():
        raise ValueError(
            "Input {} does not correspond to submission format! All predicted mask values need to be from 0 to 5!!!".format(
                filename))


    # Semantic Change Segmentation
    gt_change = np.not_equal(gt, gt_next)
    pred_change = np.not_equal(pred, pred_next)

    scs_metric.update_state(gt_change, gt_next, pred_change, pred_next, cube_id)


Found corresponding submission file /home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/predicts/load_with_tif/16N_11E-89N_2065_3647_13_PF-SR_2018-01-01.png for reference file /home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/labels/16N_11E-89N_2065_3647_13_PF-SR_2018-01-01.png
Found corresponding submission file /home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/predicts/load_with_tif/16N_11E-89N_2065_3647_13_PF-SR_2018-02-01.png for reference file /home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/labels/16N_11E-89N_2065_3647_13_PF-SR_2018-02-01.png
Found corresponding submission file /home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/predicts/load_with_tif/16N_11E-89N_2065_3647_13_PF-SR_2018-02-01.png for reference file /home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/labels/16N_11E-89N_2065_3647_13_PF-SR_2018-02-01.png
Found corresponding submi

In [5]:
scs_score = scs_metric.result()
miou_score = miou_metric.result()

# Write scores to a file named "scores.txt"
with open(os.path.join(output_dir, 'scores.txt'), 'w') as output_file:
    output_file.write("SCS: %f\n" % np.round(scs_score['SCS'] * 100, decimals=1))
    output_file.write("BC: %f\n" % np.round(scs_score['BC'] * 100, decimals=1))
    output_file.write("SC: %f\n" % np.round(scs_score['SC'] * 100, decimals=1))
    output_file.write("mIoU: %f\n" % np.round(miou_score['mIoU'] * 100, decimals=1))
    output_file.write("imp: %f\n" % np.round(miou_score['IoU_per_class'][0] * 100, decimals=1))
    output_file.write("agr: %f\n" % np.round(miou_score['IoU_per_class'][1] * 100, decimals=1))
    output_file.write("for: %f\n" % np.round(miou_score['IoU_per_class'][2] * 100, decimals=1))
    output_file.write("wet: %f\n" % np.round(miou_score['IoU_per_class'][3] * 100, decimals=1))
    output_file.write("soil: %f\n" % np.round(miou_score['IoU_per_class'][4] * 100, decimals=1))
    output_file.write("water: %f\n" % np.round(miou_score['IoU_per_class'][5] * 100, decimals=1))


In [6]:
scs_score

{'SCS': 0.07894234164330774,
 'BC': 0.06793221163779967,
 'SC': 0.08995247164881581,
 'SCS_per_cube': array([0.21673667, 0.038302  , 0.0691684 , 0.05355375, 0.11985841]),
 'BC_per_cube': [0.14574478195231266,
  0.00400289308151038,
  0.04568332232013006,
  0.004127843527653675,
  0.13269966373552758],
 'SC_per_cube': [0.28772856121644713,
  0.07260110211813445,
  0.09265348602641227,
  0.10297964775060903,
  0.10701716454505347],
 'ID_per_cube': ['16N_11E-89N_2065_3647_13_PF-SR',
  '19S_21E-313N_2528_4620_13_PF-SR',
  '20N_27E-76N_2697_3715_13_PF-SR',
  '36N_26E-144N_4881_3344_13_PF-SR',
  '36N_9E-35N_4791_3920_13_PF-SR'],
 'Length_per_cube': [23, 23, 23, 23, 23]}